# MovieBuddy — Personal AI Movie Assistant

  Step-by-step LangGraph learning project.
  Each step adds one new LangGraph concept.

In [7]:
%pip install -q langgraph langchain-openai langchain-core python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [8]:

from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END

In [9]:
# STATE - the data that flows through the nodes
class State(TypedDict):
        message: str # what user saidf
        result: str # what we responded with

In [10]:
# NODES - functions that read state and return state
def recommend_movie(state: State):
        return {"result": f"🎬 Based on your message: '{state['message']}' — I recommend Inception!"}

def lookup_movie(state: State):
        return {"result": f"🔍 Looking up info about: '{state['message']}' — Inception (2010), directed by Nolan."}

def greet(state: State):
    return {"message": state["message"] + " Welcome, I am movie buddy!!"}

def classify(state: State) -> str:
      """Route based on what the user wants."""
      msg = state["message"].lower()
      if any(word in msg for word in ["suggest", "recommend", "what should"]):
          return "recommend_movie"
      return "lookup_movie"

In [11]:
# BUILd the graph
builder = StateGraph(State)

#add nodes
builder.add_node("recommend_movie", recommend_movie)
builder.add_node("lookup_movie", lookup_movie)
builder.add_node("greet", greet)


#add edges
builder.add_edge(START, "greet")
builder.add_conditional_edges("greet", classify)
builder.add_edge("recommend_movie", END)
builder.add_edge("lookup_movie", END)

#build the graph
graph = builder.compile()



In [12]:
result = graph.invoke({"message": "Suggest me a good movie"})
print(result["result"])

🎬 Based on your message: 'Suggest me a good movie Welcome, I am movie buddy!!' — I recommend Inception!


In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.graph import MessagesState, StateGraph, START, END
from langchain_core.messages import HumanMessage, SystemMessage

In [14]:
load_dotenv()

True

In [15]:
llm = ChatOpenAI(model = 'gpt-4o-mini')
print('LLM Ready:', llm.model_name)

LLM Ready: gpt-4o-mini


In [16]:
def chatbot(state: MessagesState):
        response = llm.invoke(state["messages"])
        return {"messages": [response]}

In [17]:
builder = StateGraph(MessagesState)
builder.add_node("chatbot", chatbot)
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)
graph = builder.compile()

In [18]:
result = graph.invoke({"messages": [HumanMessage(content = "I want to watch a movie tonight")]})
print(result["messages"][-1].content)


Great! What kind of movie are you in the mood for? Here are a few genres to consider:

1. **Action/Adventure** - Fast-paced thrills and excitement.
2. **Comedy** - Something light-hearted to make you laugh.
3. **Drama** - Intense stories that evoke strong emotions.
4. **Horror** - For something scary or suspenseful.
5. **Romance** - Heartwarming stories about love.
6. **Science Fiction/Fantasy** - Exploring other worlds or imaginative concepts.
7. **Documentary** - Insightful and informative films about real-life topics.

Let me know your preference, and I can suggest some titles!


## Step 3: Add Tools (Router Pattern)      

In [19]:
from langchain_core.tools import tool                                                                     
from langgraph.prebuilt import ToolNode, tools_condition

In [20]:
@tool
def search_movie(title: str) -> str:
        """Search for a movie by title and get its details."""
        movies = {
                "inception": "Inception (2010) — Dir: Christopher Nolan. A thief who steals corporate secrets through dreams is given a chance to erase his criminal history.",
                "interstellar": "Interstellar (2014) — Dir: Christopher Nolan. A team of explorers travel through a wormhole in space to ensure humanity's survival.",
                "the dark knight": "The Dark Knight (2008) — Dir: Christopher Nolan. Batman faces the Joker, a criminal mastermind who wants to plunge Gotham into anarchy.",
        }
        return movies.get(title.lower(), f"Movie '{title}' not found in database.")

In [21]:
@tool
def get_ratings(title: str) -> str:
        """Get ratings for a movie."""
        ratings = {
                "inception": "IMDB: 8.8 | Rotten Tomatoes: 87%",
                "interstellar": "IMDB: 8.7 | Rotten Tomatoes: 73%",
                "the dark knight": "IMDB: 9.0 | Rotten Tomatoes: 94%",
        }
        return ratings.get(title.lower(), f"No ratings found for '{title}'.")

In [22]:
tools = [search_movie, get_ratings]
llm_with_tools = ChatOpenAI(model = 'gpt-4o-mini').bind_tools(tools)

In [23]:
def assistant(state: MessagesState):
        return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [29]:
builder = StateGraph(MessagesState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition)
builder.add_edge("tools", "assistant")

graph3 = builder.compile()

In [30]:
result = graph3.invoke({"messages": [HumanMessage(content= "Seach for inception and get its ratings")]})

In [31]:
for msg in result["messages"]:
        msg.pretty_print()

================================ Human Message =================================

Seach for inception and get its ratings
================================== Ai Message ==================================
Tool Calls:
  search_movie (call_9ob045YdoZ8GRIcBEEUjmF1M)
 Call ID: call_9ob045YdoZ8GRIcBEEUjmF1M
  Args:
    title: Inception
  get_ratings (call_196d3zaSVB3UWoK9ruaDIu2d)
 Call ID: call_196d3zaSVB3UWoK9ruaDIu2d
  Args:
    title: Inception
================================= Tool Message =================================
Name: search_movie

Inception (2010) — Dir: Christopher Nolan. A thief who steals corporate secrets through dreams is given a chance to erase his criminal history.
================================= Tool Message =================================
Name: get_ratings

IMDB: 8.8 | Rotten Tomatoes: 87%
================================== Ai Message ==================================

**Inception (2010)**  
- **Director:** Christopher Nolan  
- **Synopsis:** A thief who steals 

In [32]:
test_response = llm_with_tools.invoke([HumanMessage(content="Search for Inception")])
print("Content:", test_response.content)
print("Tool calls:", test_response.tool_calls)

Content: 
Tool calls: [{'name': 'search_movie', 'args': {'title': 'Inception'}, 'id': 'call_85omFUZsRl4h4THoW3yYGq26', 'type': 'tool_call'}]


In [33]:
test_response2 = llm_with_tools.invoke([HumanMessage(content="What genre is comedy?")])
print("Content:", test_response2.content)
print("Tool calls:", test_response2.tool_calls)

Content: Comedy is a genre of literature, film, and other forms of art that aims to entertain and amuse the audience through humor. It often features exaggerated characters, situations, and dialogues that provoke laughter. Common elements of comedy include misunderstandings, witty remarks, slapstick humor, and comedic timing. It can include various sub-genres, such as romantic comedy, dark comedy, satirical comedy, and farce, among others. The primary goal of comedy is to evoke a lighthearted response and provide enjoyment.
Tool calls: []


## Step 5: Memory (MemorySaver + thread_id)

In [34]:
from langgraph.checkpoint.memory import MemorySaver

In [37]:
memory = MemorySaver()
graph5 = builder.compile(checkpointer=memory)
print("Graph with memory ready!")


Graph with memory ready!


In [38]:
config = {"configurable": {"thread_id": "user_praveen_session_1"}}

In [44]:
  # Turn 1
print("=== Turn 1 ===")
result = graph5.invoke(
{"messages": [HumanMessage(content="Search for Interstellar")]},
config
)
print(result["messages"][-1].content)

=== Turn 1 ===
"Interstellar" is a 2014 film directed by Christopher Nolan. It tells the story of a team of explorers who travel through a wormhole in space to ensure humanity's survival.


In [45]:
print("\n=== Turn 2 (follow-up question) ===")
result = graph5.invoke(
{"messages": [HumanMessage(content="What were its ratings?")]},
config  # SAME thread_id = same memory
)
print(result["messages"][-1].content)


=== Turn 2 (follow-up question) ===
The ratings for "Interstellar" are as follows:
- IMDb: 8.7
- Rotten Tomatoes: 73%


In [46]:
print("\n=== Turn 3 ===")
result = graph5.invoke(
{"messages": [HumanMessage(content="Who directed it?")]},
config
)
print(result["messages"][-1].content)


=== Turn 3 ===
"Interstellar" was directed by Christopher Nolan.


In [47]:
config2 = {"configurable": {"thread_id": "user_alice_session_1"}}

result = graph5.invoke(
{"messages": [HumanMessage(content="What movie were we talking about?")]},
config2
)
print(result["messages"][-1].content)

I still don't have any information about the previous conversation regarding a movie. Could you please provide me with the name of the movie or any details related to it?


## Step 6: Custom State (Pydantic + Reducers)

In [48]:
import operator
from typing import Annotated
from pydantic import BaseModel, field_validator

In [49]:
class MovieState(MessagesState):
        favorite_genre: str
        watchlist: Annotated[list, operator.add]

In [50]:
class ValidatedMovieState(BaseModel):
        favorite_genre: str 
        watchlist: list

        @field_validator('favorite_genre')
        def validate_genre(cls, v):
                allowed = ['action', 'comedy', 'drama', 'horror', 'romance', 'sci-fi', 'thriller']
                if v and v.lower() not in allowed:
                        raise ValueError(f"Genre must be one of {allowed}, got '{v}'")
                return v.lower() if v else ''
                
                

In [55]:
def add_to_watchlist(state: MovieState):
        return {"watchlist": ["Inception"]}

def add_another(state: MovieState):
        return {"watchlist": ["Interstellar"]}

In [56]:
test_builder = StateGraph(MovieState)

In [57]:
from modulefinder import test


test_builder.add_node("add1", add_to_watchlist)
test_builder.add_node("add2", add_another)

test_builder.add_edge(START, "add1")
test_builder.add_edge("add1", "add2")
test_builder.add_edge("add2", END)

test_graph = test_builder.compile()


In [58]:
result = test_graph.invoke({"messages": [], "favorite_genre": "", "watchlist": []})
print("Watchlist:", result["watchlist"])

Watchlist: ['Inception', 'Interstellar']


In [59]:
try:
        state = ValidatedMovieState(favorite_genre="sci-fi", watchlist=["Inception"])
        print("Valid", state.favorite_genre)
except ValueError as e:
        print("Invalid:", e)






Valid sci-fi


In [60]:
try:
        state = ValidatedMovieState(favorite_genre="swestren", watchlist=["Inception"])
        print("Valid", state.favorite_genre)
except ValueError as e:
        print("Invalid:", e)



Invalid: 1 validation error for ValidatedMovieState
favorite_genre
  Value error, Genre must be one of ['action', 'comedy', 'drama', 'horror', 'romance', 'sci-fi', 'thriller'], got 'swestren' [type=value_error, input_value='swestren', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error



  ## Step 9: Streaming (Real-time Updates)

In [66]:
  print("=== Stream mode: values (see state after each node) ===\n")

  config = {"configurable": {"thread_id": "stream_demo"}}

=== Stream mode: values (see state after each node) ===



In [67]:
  for event in graph5.stream(
      {"messages": [HumanMessage(content="Search for Inception")]},
      config,
      stream_mode="values"
  ):
      # event is the full state after a node runs
      if "messages" in event:
          last_msg = event["messages"][-1]
          print(f"[{last_msg.type}] {last_msg.content[:80] if last_msg.content else '(tool call)'}...")
          print()

[human] Search for Inception...

[ai] (tool call)...

[tool] Inception (2010) — Dir: Christopher Nolan. A thief who steals corporate secrets ...

[ai] **Inception** (2010)  
**Director:** Christopher Nolan  
**Synopsis:** A thief w...



In [70]:
print("=== Stream mode: messages (token-by-token streaming) ===\n")

config2 = {"configurable": {"thread_id": "stream_demo_2"}}

for event in graph5.stream(
{"messages": [HumanMessage(content="Tell me about Interstellar in detail")]},
config2,
stream_mode="messages"
):
        # event is a tuple: (message_chunk, metadata)
        msg, metadata = event
        if msg.content:
                print(msg.content, end="", flush=True)  # Real-time typing effect!
print("\n\nDone!")

=== Stream mode: messages (token-by-token streaming) ===

**Interstellar** (2014) is a critically acclaimed science fiction film directed by Christopher Nolan. Here’s a detailed overview of the film, including its plot, themes, cast, production, ratings, and impact.

### Overview
- **Release Date**: November 7, 2014
- **Director**: Christopher Nolan
- **Screenplay**: Jonathan Nolan and Christopher Nolan
- **Producer**: Emma Thomas, Christopher Nolan, and Lynda Obst
- **Cinematography**: Hoyte van Hoytema
- **Music**: Hans Zimmer

### Plot Summary
Set in a dystopian future, Earth is facing severe ecological crises, including blight and dust storms, which threaten humanity's survival. The film follows Cooper (played by Matthew McConaughey), a former NASA pilot turned farmer, who is recruited by NASA’s original director, Professor Brand (played by Michael Caine), for a mission to find a new habitable planet.

Cooper joins a team of astronauts including Amelia Brand (Anne Hathaway) and a c


  ## Step 10: Human-in-the-Loop (Breakpoints)

In [71]:
@tool
def add_to_watchlist_tool(title: str) -> str:
        """Add a movie to the user's watchlist."""
        return f"Added '{title}' to your watchlist!"

In [72]:
tools_with_watchlist = [search_movie, get_ratings, add_to_watchlist_tool]
llm_with_approval_tools = ChatOpenAI(model="gpt-4o-mini").bind_tools(tools_with_watchlist)

In [73]:
def assistant_with_approval(state: MessagesState):
        return {"messages": [llm_with_approval_tools.invoke(state["messages"])]}

In [81]:
builder = StateGraph(MessagesState)
builder.add_node("assistant", assistant_with_approval)
builder.add_node("tools", ToolNode(tools_with_watchlist))

builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition)
builder.add_edge("tools", END)

graph10 = builder.compile(
        checkpointer=MemorySaver(),
        interrupt_before=['tools']
)

In [82]:
config = {"configurable": {"thread_id": "user_praveen_session_2"}}

In [83]:
for event in graph10.stream(
        {"messages": [HumanMessage(content="Add Inception to my watchlist")]},
        config
):
       print(f"Event: {list(event.keys())}")

Event: ['assistant']
Event: ['__interrupt__']


In [84]:
state = graph10.get_state(config)
print(f"\nPaused at: {state.next}")  # ('tools',)
print(f"Tool call: {state.values['messages'][-1].tool_calls}")

print("\n=== Step 2: User approves — resume ===")
for event in graph10.stream(None, config):  # None = continue from checkpoint
        if "messages" in event:
                print(event["messages"][-1].content if event["messages"][-1].content else "(tool executed)")


Paused at: ('tools',)
Tool call: [{'name': 'add_to_watchlist_tool', 'args': {'title': 'Inception'}, 'id': 'call_5tEfFQU1tJlveIv9yP7tjQAs', 'type': 'tool_call'}]

=== Step 2: User approves — resume ===


## 🚀 Production Agent Roadmap: Beyond LangGraph

LangGraph gives you the **orchestration foundation**, but production agents need more:

### ✅ **What You've Mastered (LangGraph Core)**
- State management (TypedDict, MessagesState, custom states)
- Node functions and conditional routing
- Tools integration (ToolNode, tools_condition)
- Memory/checkpointing (MemorySaver, thread_id)
- Streaming (real-time updates)
- Human-in-the-loop (interrupts, approvals)
- Summarization patterns

### 📚 **Essential Additions for Production**

#### 1. **Advanced LangGraph Patterns**
- **Multi-agent systems**: Agent-to-agent communication
- **Subgraphs**: Nested workflows for complex tasks
- **Error handling & retries**: Graceful failure recovery
- **Timeouts & rate limiting**: Prevent runaway agents
- **Advanced checkpointing**: PostgreSQL, MongoDB persistence (not just MemorySaver)

#### 2. **Observability & Monitoring**
- **LangSmith**: Tracing, debugging, cost tracking
- **Logging**: Structured logging (Python `logging`, `structlog`)
- **Metrics**: Prometheus, custom metrics
- **Error tracking**: Sentry, Rollbar

#### 3. **RAG (Retrieval-Augmented Generation)**
- **Vector databases**: Pinecone, Weaviate, Qdrant, Chroma
- **Embeddings**: OpenAI, Cohere, local models
- **Document loaders**: PDF, web scraping, databases
- **Chunking strategies**: Recursive, semantic, hybrid

#### 4. **Production Infrastructure**
- **API frameworks**: FastAPI, Flask (for agent endpoints)
- **Authentication**: OAuth, API keys, JWT
- **Database persistence**: PostgreSQL, MongoDB (for checkpoints + data)
- **Caching**: Redis (for performance)
- **Message queues**: RabbitMQ, Kafka (for async processing)
- **Containerization**: Docker, Kubernetes

#### 5. **Security & Safety**
- **Input validation**: Pydantic models, sanitization
- **Output filtering**: Content moderation
- **Rate limiting**: Prevent abuse
- **Secrets management**: Environment variables, Vault
- **Audit logging**: Track all agent actions

#### 6. **Testing & Quality**
- **Unit tests**: pytest for node functions
- **Integration tests**: Test full graph workflows
- **Mock LLM calls**: Use `unittest.mock` or LangChain's test utilities
- **E2E testing**: Test complete user journeys

#### 7. **Cost & Performance Optimization**
- **Token counting**: Track usage per request
- **Caching**: Cache LLM responses when appropriate
- **Model selection**: Use cheaper models when possible
- **Batch processing**: Process multiple requests efficiently
- **Async operations**: Use `asyncio` for concurrent tasks

### 🎯 **Recommended Learning Path**

1. **Next Steps (Week 1-2)**
   - LangSmith integration (observability)
   - PostgreSQL checkpointing (persistent memory)
   - Error handling patterns

2. **Intermediate (Week 3-4)**
   - RAG implementation (vector DB + embeddings)
   - FastAPI wrapper for your agent
   - Basic monitoring setup

3. **Advanced (Month 2+)**
   - Multi-agent systems
   - Production deployment (Docker, cloud)
   - Performance optimization

### 📖 **Key Resources**
- **LangGraph Docs**: https://langchain-ai.github.io/langgraph/
- **LangSmith**: https://docs.smith.langchain.com/
- **LangChain Cookbook**: Advanced patterns
- **Production Patterns**: LangChain's production guide

### 💡 **Bottom Line**
LangGraph = **Orchestration Engine** ✅  
Production Agent = LangGraph + **RAG + Observability + Infrastructure + Security** 🚀

You have the foundation! Now layer on the production essentials.
